# Creating a test data set with similar properties as an original population

In [1]:
import numpy as np
import pandas as pd
import pprint
import random
random.seed(42)

## Create the original population
Each individual has four properties:
- age (numeric)
- civil_status (categorical)
- personal_status (categorical)
- pays_taxes (boolean)

These properties are assigned independently and in a random fashion. The status fields have the following values with their fractions in the whole population (unconditional probabilities) -- just some arbitrary values to initialize the data). Note that the assigned "probabilities" do not actually have to sum up to 1, they will be normalized by the `get_random_status` function; in this way they can also just be counts in the population. This will come in handy later on.

In [2]:
civil_status_values = {0: 0.3, 1: 0.2, 2: 0.5}
personal_status_values = {0: 1, 1: 3, 2: 2, 3: 4}

We need a function that allows us to randomly assign status values according to given probabilities; in the initial population these probabilities are given by the fractions that we specified above.

In [3]:
def get_random_status(status_values):
    status = list(status_values.keys())
    prob = np.array([0.0] + list(status_values.values()))
    prob /= sum(prob) # normalize, so that the last entry of the cumsum will be 1, the range of random
    prob_sum = np.cumsum(prob)
    return status[np.searchsorted(prob_sum, random.random()) - 1]
# test the function; note that due to random input the result will not always be exactly 40:80
tst = {0: 0, 1: 0} # initialize to zero, then add in loop for count of each value
for i in range(120):
    tst[get_random_status({0: 33, 1: 67})] += 1
print(tst)

{0: 45, 1: 75}


In [4]:
population_size = 12375
population = []
for i in range(population_size):
    population.append({
        'age': max(1, int(random.gauss(mu=32.5, sigma=7.2))),
        'civil_status': get_random_status(civil_status_values),
        'personal_status': get_random_status(personal_status_values),
        'pays_taxes': int(random.random() < 0.57)
        })
df_population = pd.DataFrame(population)
df_population.describe()

,age,civil_status,pays_taxes,personal_status
count,12375.000000,12375.000000,12375.000000,12375.000000
mean,31.971879,1.205657,0.577616,1.899313
std,7.250534,0.870451,0.493959,1.043368
min,4.000000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,0.000000,1.000000
50%,32.000000,2.000000,1.000000,2.000000
75%,37.000000,2.000000,1.000000,3.000000
max,58.000000,2.000000,1.000000,3.000000


## Analyze the original population
From now on, the above `population` is the "truth" and the task is to create a synthetic population (of a different size) that statistically resembles the original one.

We do this by defining a tree with one of the "features" of each person at every level. Then we find the conditional probabilities for each branch. The ordering of the features is arbitrary. For example:

                                     person
                                        |
                    --------------  pays_taxes  -----------------
                    |                                           |
                 1=True ()                                   0=False ()
                    |                                           |
        ------ civil_status ------                  ------ civil_status ------
        |           |            |                  |           |            |
       0 ()        1 ()         2 ()               0 ()        1 ()         2 ()

Below this level, the `person_status` would be the next level of branches. The parenthesis represent the conditional probabilities, e.g. for `civil_status` being `0/1/2` _given that_ `pays_taxes = True`.

Note that in our sample population, the different properties were assigned independent of each other; therefore the conditional probabilities for `civil_status` in the `True` branch will only differ marginally from those in the `False` branch.

But first we find the probabilities for the first property, `pays_taxes`:

In [5]:
pop__pays_taxes = dict(df_population.groupby('pays_taxes')['pays_taxes'].count())
print(pop__pays_taxes) 

{0: 5227, 1: 7148}


Now we look at the `civil_status` in each branch under `pays_taxes`:

In [6]:
pop = {}
pop['pays_taxes'] = {}
for pt in [0, 1]:
    tmp = dict(df_population[df_population.pays_taxes == pt].groupby(
                   'civil_status')['civil_status'].count())
    pop['pays_taxes'][pt] = {
        'count': pop__pays_taxes[pt],
        'civil_status': {k: {'count': v, 'personal_status': {}} for k, v in tmp.items()}
    }

In [7]:
pprint.pprint(pop)

{'pays_taxes': {0: {'civil_status': {0: {'count': 1549, 'personal_status': {}},
                                     1: {'count': 1063, 'personal_status': {}},
                                     2: {'count': 2615, 'personal_status': {}}},
                    'count': 5227},
                1: {'civil_status': {0: {'count': 2128, 'personal_status': {}},
                                     1: {'count': 1413, 'personal_status': {}},
                                     2: {'count': 3607, 'personal_status': {}}},
                    'count': 7148}}}


We can see that the `count` values for `civil_status` being `0/1/2` add up to the `count` value in the next higher branch, of `pays_taxes`. It would be easy to transform everything to probabilities, but it's not necessary given that the function `get_random_status` can work with counts.

Let's add one more layer.

In [17]:
for pt in [0, 1]:
    for cs in civil_status_values.keys():
        # Note: We restrict the population to one combination of pays_taxes and civil_status at a
        #       time and then group by the personal_status. In this way we effectively find
        #       conditional probabilities like 
        #       P(personal_status==x | pays_taxes==pt && civil_status == cs)
        #       just that we record them as counts (which always sum up to the one at the next higher
        #       level rather than to 1).
        tmp = dict(df_population[(df_population.pays_taxes == pt)&
                                 (df_population.civil_status == cs)].groupby(
                   'personal_status')['personal_status'].count())
        pop['pays_taxes'][pt]['civil_status'][cs]['personal_status'] = {
            k: {'count': v, 'age': {}} for k, v in tmp.items()}

In [18]:
pprint.pprint(pop)

{'pays_taxes': {0: {'civil_status': {0: {'count': 1549,
                                         'personal_status': {0: {'age': {},
                                                                 'count': 166},
                                                             1: {'age': {},
                                                                 'count': 478},
                                                             2: {'age': {},
                                                                 'count': 291},
                                                             3: {'age': {},
                                                                 'count': 614}}},
                                     1: {'count': 1063,
                                         'personal_status': {0: {'age': {},
                                                                 'count': 118},
                                                             1: {'age': {},
                              

Since the age is stored as `int` one can in principle use the same procedure to fill in the `age` branches -- there will be one category for each age in years, and at this level of detail there are still hundreds of persons in each leaf. If there are too few (e.g. if many more branching levels are used) then one should possible derive an age distribution per leaf and work with that.

But how can we use the above to create a synthetic data set?

## Synthesize a similar population

In [19]:
synthetic_population_size = int(population_size / 3)

In [22]:
df_synthetic = pd.DataFrame(np.zeros((synthetic_population_size, 4)), columns=df_population.columns)
for i in range(synthetic_population_size):
    # In assigning the values of an artificial persion, we have to follow the branching sequence
    # of our tree:
    df_synthetic.loc[i, 'pays_taxes'] = get_random_status(
        { pt: pop['pays_taxes'][pt]['count'] for pt in [0, 1]} )

    # For the next property, we have to use the counts (probabilities) in the appropriate branch
    # below pays_taxes, based on the value found above.
    df_synthetic.loc[i, 'civil_status'] = get_random_status(
        {cs: pop['pays_taxes'][df_synthetic.loc[i, 'pays_taxes']]['civil_status'][cs]['count']
         for cs in civil_status_values.keys()})

    # And finally, yet another layer deeper in the data structure for personal_status.
    df_synthetic.loc[i, 'personal_status'] = get_random_status(
        {ps: pop['pays_taxes'][df_synthetic.loc[i, 'pays_taxes']]
                ['civil_status'][df_synthetic.loc[i, 'civil_status']]
                ['personal_status'][ps]['count']
         for ps in personal_status_values.keys()})
    
df_synthetic.describe(percentiles=[])

,age,civil_status,pays_taxes,personal_status
count,4125.0,4125.000000,4125.000000,4125.000000
mean,0.0,1.222545,0.572121,1.887030
std,0.0,0.864442,0.494831,1.041496
min,0.0,0.000000,0.000000,0.000000
50%,0.0,2.000000,1.000000,2.000000
max,0.0,2.000000,1.000000,3.000000


In [23]:
df_population.describe(percentiles=[]) # for comparison: the original population;
                                       # compare the mean and std; age has not been set

,age,civil_status,pays_taxes,personal_status
count,12375.000000,12375.000000,12375.000000,12375.000000
mean,31.971879,1.205657,0.577616,1.899313
std,7.250534,0.870451,0.493959,1.043368
min,4.000000,0.000000,0.000000,0.000000
50%,32.000000,2.000000,1.000000,2.000000
max,58.000000,2.000000,1.000000,3.000000
